# Integrated Gradients calculations

## Set-up

In [1]:
from modeling.pretrained_bert import PretrainedBertModule
from trainer import load_model
import torch
from torch.utils.data import DataLoader

from feature_importance import IntegratedGradients

## Loading models

In [2]:
bert_model_path = "mqnli_models/bert-easy-best.pt"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

bert_model, _ = load_model(PretrainedBertModule, bert_model_path, device=device)

bert_model = bert_model.to(device)
bert_model.eval() # disable dropout

None

## Loading data (Bert)

In [3]:
bert_easy_data_path = "mqnli_models/bert-preprocessed-data.pt"

bert_data = torch.load(bert_easy_data_path)

bert_dev_set = bert_data.dev

## Calculations

In [4]:
def analyze_sample(model, examples, n=8, batch_size=4, output_filename=None, layer=None, shuffle=True):
    n_batches = int(n / batch_size)
    ig = IntegratedGradients(model, layer=layer)
    dataloader = DataLoader(examples, batch_size=batch_size, shuffle=shuffle)
    data = []
    with torch.no_grad():
        for i, input_tuple in enumerate(dataloader, start=1):
            if i % 100 == 0:
                print(f"Batch {i} of {n_batches}")
            input_tuple = tuple([x.to(device) for x in input_tuple])
            data += ig.predict_with_ig(tuple(input_tuple))
            if i == n_batches:
                break
    if output_filename:
        ig.to_json(data, output_filename)
    return data

### Big experiment

In [5]:
#data = analyze_sample(bert_model, bert_dev_set, n=20000, output_filename='ig-batch01.json')

In [6]:
#IntegratedGradients.visualize(data[: 5])

### Demos

In [7]:
data_emb = analyze_sample(bert_model, bert_dev_set, shuffle=False)

In [8]:
IntegratedGradients.visualize(data_emb[: 3])

In [13]:
data_enc0 = analyze_sample(bert_model, bert_dev_set, shuffle=False, layer=bert_model.bert.encoder.layer[0])

In [14]:
IntegratedGradients.visualize(data_enc0[: 3])

In [15]:
data_enc11 = analyze_sample(bert_model, bert_dev_set, shuffle=False, layer=bert_model.bert.encoder.layer[11])

In [17]:
IntegratedGradients.visualize(data_enc[: 3])